**So I have my MVP but now I need to add more features to my dataset. The plan is to add the following features:**
1. position
1. height and weight
1. right handed or left
1. went to college or no?
1. Debut age?
1. change strikeout to non-strikeout rate?
1. add more MLB players?

In [276]:
import pickle
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import math
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from collections import defaultdict 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [3]:
# lets load in the current dataframe with all my data
with open('total_players_2000.pkl', 'rb') as handle:
    wow = pickle.load(handle)

In [4]:
wow

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,MINORS_BA,MINORS_OPS,MLB_G,MLB_AB,MINORS_R_per_AB,MINORS_SO_per_AB,MINORS_BB_per_AB
0,Evert-Jean 't Hoen,/register/player.fcgi?id=thoen-001eve,485,1576,180,326,38,176,145,458,0.207,0.622,0,0,0.114213,0.290609,0.092005
1,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,0.284,0.833,46,78,0.156048,0.205326,0.093828
2,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,0.259,0.660,79,218,0.150238,0.164149,0.081479
3,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,0.278,0.725,191,390,0.131893,0.131434,0.088695
4,Edgard Clemente,/register/player.fcgi?id=clemen001edg,973,3586,501,963,125,496,285,875,0.269,0.773,114,257,0.139710,0.244004,0.079476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,Geoff Blum,/register/player.fcgi?id=blum--001geo,602,2103,302,564,28,260,264,302,0.268,0.728,1389,3966,0.143604,0.143604,0.125535
2383,Wilton Guerrero,/register/player.fcgi?id=guerre001wil,732,2816,404,877,11,249,174,375,0.311,0.740,678,1678,0.143466,0.133168,0.061790
2384,Mike Mordecai,/register/player.fcgi?id=mordec001mic,564,2007,299,538,33,227,212,306,0.268,0.718,793,1362,0.148979,0.152466,0.105630
2385,Terry Jones,/register/player.fcgi?id=jones-006ter,720,2782,449,763,4,193,225,482,0.274,0.651,227,530,0.161395,0.173257,0.080877


# Web scraping 

In [249]:
url = 'https://www.baseball-reference.com' + str(wow.iloc[10,1])
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html class="no-js" data-root="/home/br/www" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202004133" rel="dns-prefetch"/>
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[1].replace(/\+/g,' '));}; document.srdev = gup('srdev')</script>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d3k2oh6evki4b7.cloudfront.net" rel="preconnect"/>
<style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvetic

In [250]:
#testing scrape code
info_dict = defaultdict(str)
for tag in soup.find_all('div', itemtype="https://schema.org/Person"):
    for i,tag2 in enumerate(tag.find_all('p')):
        if tag2.strong is not None:
            if tag2.strong.text == "Positions:":
                info_dict["Position"] = tag2.text.replace('\n','').split(':',1)[1].strip().split(' ',1)[0].replace(',','')
            elif tag2.strong.text == "Position:":
                info_dict["Position"] = tag2.text.split(':',1)[1].strip()
            elif tag2.strong.text == "Schools:":
                info_dict["Schools"] = tag2.text.replace('\n','').split(':',1)[1]
            elif tag2.strong.text == "School:":
                info_dict["Schools"] = tag2.text.split(':',1)[1].strip()
            elif tag2.strong.text == "Bats: ":
                info_dict["Bats"] = tag2.text.replace('\n','').split(':',1)[1][0:6].strip()
            elif tag2.strong.text == "Debut:":
                info_dict["Debut"] = tag2.text.replace('\n','').split(':',1)[1].split('(',1)[0].strip()
            elif tag2.strong.text == "Born:":
                info_dict["Born"] = tag2.text.replace('\n','').split(':',1)[1][0:21].strip()
            elif tag2.strong.text == "Draft":
                index_num = tag2.text.find('round',len(tag2.text) - 100)
                info_dict["Draft"] = tag2.text[index_num-5:index_num-3].strip()
        else:
            for i,tag3 in enumerate(tag2.find_all('span')):
                if i ==0:
                    info_dict['Height'] = tag3.text
                if i == 1:
                    info_dict['Weight'] = tag3.text
                              
info_dict


defaultdict(str,
            {'Position': 'Outfielder',
             'Bats': 'Both',
             'Height': '5-10',
             'Weight': '175lb',
             'Born': 'March 11, 1967',
             'Draft': 'Dr',
             'Schools': 'Pasadena City College (Pasadena, CA)',
             'Debut': 'September 1, 1995'})

In [209]:
#adding more columns for all this data
wow['Position'] = ''
wow['Bats'] = ''
wow['Height'] = ''
wow['Weight'] = ''
wow['Born'] = ''
wow['Debut'] = ''
wow['Draft'] = ''
wow['School'] = ''

In [259]:
#more testing of scraping
for tag in soup.find_all('div', itemtype="https://schema.org/Person"):
    for i,tag2 in enumerate(tag.find_all('p')):
        if tag2.strong is not None:
            if tag2.strong.text == "Draft":
                index_num = tag2.text.find('round')
                print(tag2.text[index_num-5:index_num-3].strip())
                #info_dict["Draft"] = tag2.text[index_num-5:index_num+5].strip()

13


In [260]:
#lets write a function that gets all the additional features that I want for each player
def grab_more_features(df):
    """
    Arguments: a dataframe with baseball players and url paths already in there
    Returns: the same dataframe but with more information about them like their Position, if they went to college, 
    the round they were drafted in, batting right-handed or left-handed, when they got to the pros, when they were born,
    their height, and their weight. 
    """
    
    for index,row in enumerate(df.values.tolist()):
        
        #scraping the specific player's stat page
        url = 'https://www.baseball-reference.com' + str(row[1])
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        #initializing a dictionary to put the data in
        info_dict = defaultdict(str)
        
        #looping through the scraped jumble for exactly the data we want
        for tag in soup.find_all('div', itemtype="https://schema.org/Person"):
            for i,tag2 in enumerate(tag.find_all('p')):
                if tag2.strong is not None:
                    if tag2.strong.text == "Positions:":
                        info_dict["Position"] = tag2.text.replace('\n','').split(':',1)[1].strip().split(' ',1)[0].replace(',','')
                    elif tag2.strong.text == "Position:":
                        info_dict["Position"] = tag2.text.split(':',1)[1].strip()
                    elif tag2.strong.text == "Schools:":
                        info_dict["Schools"] = tag2.text.replace('\n','').split(':',1)[1]
                    elif tag2.strong.text == "School:":
                        info_dict["Schools"] = tag2.text.split(':',1)[1].strip()
                    elif tag2.strong.text == "Bats: ":
                        info_dict["Bats"] = tag2.text.replace('\n','').split(':',1)[1][0:6].strip()
                    elif tag2.strong.text == "Debut:":
                        info_dict["Debut"] = tag2.text.replace('\n','').split(':',1)[1].split('(',1)[0].strip()
                    elif tag2.strong.text == "Born:":
                        info_dict["Born"] = tag2.text.replace('\n','').split(':',1)[1][0:21].strip()
                    elif tag2.strong.text == "Draft":
                        index_num = tag2.text.find('round')
                        info_dict["Draft"] = tag2.text[index_num-5:index_num-3].strip()
                else:
                    for j,tag3 in enumerate(tag2.find_all('span')):
                        if j ==0:
                            info_dict['Height'] = tag3.text
                        if j == 1:
                            info_dict['Weight'] = tag3.text
        
        #assigning the data we grabbed to the specific cells in the dataframe
        df['Position'][index] = info_dict['Position']
        df['Bats'][index] = info_dict['Bats']
        df['Height'][index] = info_dict['Height']
        df['Weight'][index] = info_dict['Weight']
        df['Born'][index] = info_dict['Born']
        df['Debut'][index] = info_dict['Debut']
        if info_dict['Draft']:
            df['Draft'][index] = int(info_dict['Draft'])
        if info_dict['Schools']:
            df['School'][index] = 1
        else:
            df['School'][index] = 0
        print(str(index) + " completed!")
            
    return df

In [261]:
#running the above function
full_df = grab_more_features(wow) 
full_df

/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samir/opt/anaconda3/envs

0 completed!
1 completed!
2 completed!
3 completed!


/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4 completed!
5 completed!
6 completed!
7 completed!
8 completed!
9 completed!
10 completed!
11 completed!
12 completed!
13 completed!
14 completed!
15 completed!
16 completed!
17 completed!
18 completed!
19 completed!
20 completed!
21 completed!
22 completed!
23 completed!
24 completed!
25 completed!
26 completed!
27 completed!
28 completed!
29 completed!
30 completed!
31 completed!
32 completed!
33 completed!
34 completed!
35 completed!
36 completed!
37 completed!
38 completed!
39 completed!
40 completed!
41 completed!
42 completed!
43 completed!
44 completed!
45 completed!
46 completed!
47 completed!
48 completed!
49 completed!
50 completed!
51 completed!
52 completed!
53 completed!
54 completed!
55 completed!
56 completed!
57 completed!
58 completed!
59 completed!
60 completed!
61 completed!
62 completed!
63 completed!
64 completed!
65 completed!
66 completed!
67 completed!
68 completed!
69 completed!
70 completed!
71 completed!
72 completed!
73 completed!
74 completed!
75 completed

557 completed!
558 completed!
559 completed!
560 completed!
561 completed!
562 completed!
563 completed!
564 completed!
565 completed!
566 completed!
567 completed!
568 completed!
569 completed!
570 completed!
571 completed!
572 completed!
573 completed!
574 completed!
575 completed!
576 completed!
577 completed!
578 completed!
579 completed!
580 completed!
581 completed!
582 completed!
583 completed!
584 completed!
585 completed!
586 completed!
587 completed!
588 completed!
589 completed!
590 completed!
591 completed!
592 completed!
593 completed!
594 completed!
595 completed!
596 completed!
597 completed!
598 completed!
599 completed!
600 completed!
601 completed!
602 completed!
603 completed!
604 completed!
605 completed!
606 completed!
607 completed!
608 completed!
609 completed!
610 completed!
611 completed!
612 completed!
613 completed!
614 completed!
615 completed!
616 completed!
617 completed!
618 completed!
619 completed!
620 completed!
621 completed!
622 completed!
623 comple

1097 completed!
1098 completed!
1099 completed!
1100 completed!
1101 completed!
1102 completed!
1103 completed!
1104 completed!
1105 completed!
1106 completed!
1107 completed!
1108 completed!
1109 completed!
1110 completed!
1111 completed!
1112 completed!
1113 completed!
1114 completed!
1115 completed!
1116 completed!
1117 completed!
1118 completed!
1119 completed!
1120 completed!
1121 completed!
1122 completed!
1123 completed!
1124 completed!
1125 completed!
1126 completed!
1127 completed!
1128 completed!
1129 completed!
1130 completed!
1131 completed!
1132 completed!
1133 completed!
1134 completed!
1135 completed!
1136 completed!
1137 completed!
1138 completed!
1139 completed!
1140 completed!
1141 completed!
1142 completed!
1143 completed!
1144 completed!
1145 completed!
1146 completed!
1147 completed!
1148 completed!
1149 completed!
1150 completed!
1151 completed!
1152 completed!
1153 completed!
1154 completed!
1155 completed!
1156 completed!
1157 completed!
1158 completed!
1159 com

1610 completed!
1611 completed!
1612 completed!
1613 completed!
1614 completed!
1615 completed!
1616 completed!
1617 completed!
1618 completed!
1619 completed!
1620 completed!
1621 completed!
1622 completed!
1623 completed!
1624 completed!
1625 completed!
1626 completed!
1627 completed!
1628 completed!
1629 completed!
1630 completed!
1631 completed!
1632 completed!
1633 completed!
1634 completed!
1635 completed!
1636 completed!
1637 completed!
1638 completed!
1639 completed!
1640 completed!
1641 completed!
1642 completed!
1643 completed!
1644 completed!
1645 completed!
1646 completed!
1647 completed!
1648 completed!
1649 completed!
1650 completed!
1651 completed!
1652 completed!
1653 completed!
1654 completed!
1655 completed!
1656 completed!
1657 completed!
1658 completed!
1659 completed!
1660 completed!
1661 completed!
1662 completed!
1663 completed!
1664 completed!
1665 completed!
1666 completed!
1667 completed!
1668 completed!
1669 completed!
1670 completed!
1671 completed!
1672 com

2123 completed!
2124 completed!
2125 completed!
2126 completed!
2127 completed!
2128 completed!
2129 completed!
2130 completed!
2131 completed!
2132 completed!
2133 completed!
2134 completed!
2135 completed!
2136 completed!
2137 completed!
2138 completed!
2139 completed!
2140 completed!
2141 completed!
2142 completed!
2143 completed!
2144 completed!
2145 completed!
2146 completed!
2147 completed!
2148 completed!
2149 completed!
2150 completed!
2151 completed!
2152 completed!
2153 completed!
2154 completed!
2155 completed!
2156 completed!
2157 completed!
2158 completed!
2159 completed!
2160 completed!
2161 completed!
2162 completed!
2163 completed!
2164 completed!
2165 completed!
2166 completed!
2167 completed!
2168 completed!
2169 completed!
2170 completed!
2171 completed!
2172 completed!
2173 completed!
2174 completed!
2175 completed!
2176 completed!
2177 completed!
2178 completed!
2179 completed!
2180 completed!
2181 completed!
2182 completed!
2183 completed!
2184 completed!
2185 com

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,MINORS_SO_per_AB,MINORS_BB_per_AB,Position,Bats,Height,Weight,Born,Debut,Draft,School
0,Evert-Jean 't Hoen,/register/player.fcgi?id=thoen-001eve,485,1576,180,326,38,176,145,458,...,0.290609,0.092005,Shortstop,Right,6-2,190lb,"November 8, 1975",,32,1
1,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,0.205326,0.093828,First,Left,6-1,195lb,"July 23, 1974","April 11, 2001",69,1
2,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,0.164149,0.081479,Shortstop,Right,5-11,175lb,"August 1, 1974","May 17, 1998",42,1
3,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,0.131434,0.088695,Catcher,Right,6-0,195lb,"September 24, 1971","May 9, 2001",9,1
4,Edgard Clemente,/register/player.fcgi?id=clemen001edg,973,3586,501,963,125,496,285,875,...,0.244004,0.079476,Outfielder,Right,5-11,188lb,"December 15, 1975","September 10, 1998",10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,Geoff Blum,/register/player.fcgi?id=blum--001geo,602,2103,302,564,28,260,264,302,...,0.143604,0.125535,Third,Both,6-3,220lb,"April 26, 1973","August 9, 1999",7,1
2383,Wilton Guerrero,/register/player.fcgi?id=guerre001wil,732,2816,404,877,11,249,174,375,...,0.133168,0.061790,Second,Both,5-11,145lb,"October 24, 1974","September 3, 1996",,0
2384,Mike Mordecai,/register/player.fcgi?id=mordec001mic,564,2007,299,538,33,227,212,306,...,0.152466,0.105630,Shortstop,Right,5-11,175lb,"December 13, 1967","May 8, 1994",33,1
2385,Terry Jones,/register/player.fcgi?id=jones-006ter,720,2782,449,763,4,193,225,482,...,0.173257,0.080877,Centerfielder,Both,5-10,165lb,"February 15, 1971","September 8, 1996",40,1


In [262]:
full_df

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,MINORS_SO_per_AB,MINORS_BB_per_AB,Position,Bats,Height,Weight,Born,Debut,Draft,School
0,Evert-Jean 't Hoen,/register/player.fcgi?id=thoen-001eve,485,1576,180,326,38,176,145,458,...,0.290609,0.092005,Shortstop,Right,6-2,190lb,"November 8, 1975",,32,1
1,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,0.205326,0.093828,First,Left,6-1,195lb,"July 23, 1974","April 11, 2001",69,1
2,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,0.164149,0.081479,Shortstop,Right,5-11,175lb,"August 1, 1974","May 17, 1998",42,1
3,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,0.131434,0.088695,Catcher,Right,6-0,195lb,"September 24, 1971","May 9, 2001",9,1
4,Edgard Clemente,/register/player.fcgi?id=clemen001edg,973,3586,501,963,125,496,285,875,...,0.244004,0.079476,Outfielder,Right,5-11,188lb,"December 15, 1975","September 10, 1998",10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,Geoff Blum,/register/player.fcgi?id=blum--001geo,602,2103,302,564,28,260,264,302,...,0.143604,0.125535,Third,Both,6-3,220lb,"April 26, 1973","August 9, 1999",7,1
2383,Wilton Guerrero,/register/player.fcgi?id=guerre001wil,732,2816,404,877,11,249,174,375,...,0.133168,0.061790,Second,Both,5-11,145lb,"October 24, 1974","September 3, 1996",,0
2384,Mike Mordecai,/register/player.fcgi?id=mordec001mic,564,2007,299,538,33,227,212,306,...,0.152466,0.105630,Shortstop,Right,5-11,175lb,"December 13, 1967","May 8, 1994",33,1
2385,Terry Jones,/register/player.fcgi?id=jones-006ter,720,2782,449,763,4,193,225,482,...,0.173257,0.080877,Centerfielder,Both,5-10,165lb,"February 15, 1971","September 8, 1996",40,1


In [263]:
#saving my progress
with open('full_feature_df.pkl', 'wb') as handle:
    pickle.dump(full_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [469]:
#aaand we're back working
with open('full_feature_df.pkl', 'rb') as handle:
    full_df = pickle.load(handle)

# Data Cleaning (and minor feature engineering)

In [470]:
full_df.Debut

0                         
1           April 11, 2001
2             May 17, 1998
3              May 9, 2001
4       September 10, 1998
               ...        
2382        August 9, 1999
2383     September 3, 1996
2384           May 8, 1994
2385     September 8, 1996
2386     September 1, 1989
Name: Debut, Length: 2387, dtype: object

In [285]:
#practicing change a date string to a datetime object
date = 'November 8, 1975'
dt.datetime.strptime(date, "%B %d, %Y")

datetime.datetime(1975, 11, 8, 0, 0)

In [471]:
#changing all the columns with dates to datetime format
full_df.Born = pd.to_datetime(full_df.Born,format="%B %d, %Y",errors='coerce',exact=False)
full_df.Debut = pd.to_datetime(full_df.Debut,format="%B %d, %Y",errors='coerce',exact=False)

In [458]:
#filling in the null values
full_df.Born = full_df.Born.fillna(0)
full_df.Debut = full_df.Debut.fillna(0)

In [497]:
#filling in the blank values
full_df.Draft = full_df.Draft.replace('', 0,)
full_df.Draft.astype(int)

In [460]:
#changing the School column to an int rather than a string
full_df.School = full_df.School.astype('int64')

In [483]:
#making a new feature called Debut Age to figure out the age that each player made it to the MLB
full_df['Debut_Age'] = (full_df.Debut - full_df.Born)/dt.timedelta(days=365)

In [491]:
#cleaning up the weight column by removing the 'lb' and changing the data type to int
full_df.Weight = full_df.Weight.str.replace('lb', '').astype(int)

**Next steps are:**
1. Turning height into inches and dtype:int
1. turning position, School and Bats into categorical variables that will show up in a regression model
1. Running basic regression model again
1. If we still need more features then let's go back and get more otherwise proceed to cross_validation/reg steps

In [502]:
with open('full_feature_df.pkl', 'wb') as handle:
    pickle.dump(full_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [533]:
#changing height from a string to inches in integers
full_df.Height = full_df.Height.apply(lambda x: (int(x.split('-')[0])*12) + int(x.split('-')[1]))

In [564]:
#cleaning up the categorical variables so we can make them dummies
full_df.loc[full_df.Position=='First', 'Position'] = 'First Baseman'
full_df.loc[full_df.Position=='Second', 'Position'] = 'Second Baseman'
full_df.loc[full_df.Position=='Third', 'Position'] = 'Third Baseman'
full_df.loc[full_df.Position=='Leftfielder', 'Position'] = 'Outfielder'
full_df.loc[full_df.Position=='Rightfielder', 'Position'] = 'Outfielder'
full_df.loc[full_df.Position=='Centerfielder', 'Position'] = 'Outfielder'

In [579]:
#changing the players who either had pinch hitter or designated hitter as their position 
#to their most commonly played field position. We want to eliminate categories with low amounts of players

full_df.at[1086, 'Position'] = 'Outfielder'
full_df.at[2064,'Position'] = 'Third Baseman'
full_df.at[2300,'Position'] = 'Outfielder'
full_df.at[77,'Position'] = 'First Baseman'
full_df.at[588,'Position'] = 'First Baseman'
full_df.at[1465,'Position'] = 'First Baseman'
full_df.at[1968,'Position'] = 'First Baseman'
full_df.at[2018,'Position'] = 'First Baseman'
full_df.at[2088,'Position'] = 'Outfielder'
full_df.at[2124,'Position'] = 'First Baseman'
full_df.at[2184,'Position'] = 'First Baseman'
full_df.at[2244,'Position'] = 'First Baseman'
full_df.at[2335,'Position'] = 'Third Baseman'
full_df.at[2373,'Position'] = 'First Baseman'

In [627]:
#ok now that this is done we can make them dummies. Let's test this to make sure it works
pd.get_dummies(full_df.Position)

,Catcher,First Baseman,Outfielder,Second Baseman,Shortstop,Third Baseman
0,0,0,0,0,1,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,1,0,0,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
2382,0,0,0,0,0,1
2383,0,0,0,1,0,0
2384,0,0,0,0,1,0
2385,0,0,1,0,0,0


In [632]:
#saving progress
with open('full_feature_df.pkl', 'wb') as handle:
    pickle.dump(full_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [729]:
#making a new df that is ready to use pd.get_dummies on
dummy_df = full_df.drop(['url_path'],axis=1)
dummy_df.index = dummy_df.player_name
dummy_df = dummy_df.drop(['player_name'],axis=1)
dummy_df = pd.get_dummies(dummy_df)

In [733]:
#changing the names of some columns so they don't have spaces
dummy_df.rename(columns={"Position_Third Baseman":"Position_Third_Baseman", "Position_First Baseman": "Position_First_Baseman", "Position_Second Baseman": "Position_Second_Baseman"}, inplace=True)

In [735]:
#turning weight and height into one feature that uses both of them:bmi
dummy_df['bmi'] = (dummy_df.Weight/2.205)/((dummy_df.Height/39.37)**2)

**Let's save our progress for now**

In [ ]:
with open('dummy_df.pkl', 'wb') as handle:
    pickle.dump(dummy_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# Final Data Collection  

**After running some basic models on my data I have one more thing I want to try. Below is some code to figure out how many seasons someone spent in the MLB. The idea is to normalize the MLB_AB column by dividing the ABs by number of seasons**

**At this point, I've also decided to focus my model on predicting MLB ABs only for players who actually made it to the MLB. The model is having trouble with more than half of the observations having 0 as the target variable because the player has no MLB ABs**

In [881]:
#making a new df for only players with MLB ABs
mlb_only = full_df[full_df['MLB_AB'] > 0]
mlb_only.reset_index(drop=True, inplace=True)
mlb_only['MLB_seasons'] = 0
mlb_only['MINORS_seasons'] = 0

/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [882]:
mlb_only

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,Bats,Height,Weight,Born,Debut,Draft,School,Debut_Age,MLB_seasons,MINORS_seasons
0,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,Left,73,195,1974-07-23,2001-04-11,69,1,26.736986,0,0
1,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,Right,71,175,1974-08-01,1998-05-17,42,1,23.808219,0,0
2,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,Right,72,195,1971-09-24,2001-05-09,9,1,29.643836,0,0
3,Edgard Clemente,/register/player.fcgi?id=clemen001edg,973,3586,501,963,125,496,285,875,...,Right,71,188,1975-12-15,1998-09-10,10,0,22.753425,0,0
4,Steve Decker,/register/player.fcgi?id=decker001ste,913,3228,458,975,101,575,412,470,...,Right,75,205,1965-10-25,1990-09-18,21,1,24.915068,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,Geoff Blum,/register/player.fcgi?id=blum--001geo,602,2103,302,564,28,260,264,302,...,Both,75,220,1973-04-26,1999-08-09,7,1,26.304110,0,0
1147,Wilton Guerrero,/register/player.fcgi?id=guerre001wil,732,2816,404,877,11,249,174,375,...,Both,71,145,1974-10-24,1996-09-03,0,0,21.876712,0,0
1148,Mike Mordecai,/register/player.fcgi?id=mordec001mic,564,2007,299,538,33,227,212,306,...,Right,71,175,1967-12-13,1994-05-08,33,1,26.419178,0,0
1149,Terry Jones,/register/player.fcgi?id=jones-006ter,720,2782,449,763,4,193,225,482,...,Both,70,165,1971-02-15,1996-09-08,40,1,25.580822,0,0


In [911]:
#practicing grabbing number of MLB/MINORS seasons
url = 'https://www.baseball-reference.com' + '/register/player.fcgi?id=baughm001jus'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

In [912]:
#practicing grabbing number of MLB seasons
for tag in soup.find_all('tfoot'):
    for tag2 in tag.find_all('tr', class_='mlb'):
        if 'season' in tag2.text:
            print(tag2.text.split('(')[1].split(' ')[0])

2


In [905]:
#practicing grabbing minors seasons
for tag3 in soup.find_all('tfoot'):
    if tag3.find('tr', class_='minors'):
        for tag4 in tag3.find('tr', class_='minors'):
            if 'seasons' in tag4.text:
                print(tag4.text.split('(')[1].split(' ')[0])

12


In [845]:
#more testing
for index,row in enumerate(mlb_only.values.tolist()):
    url = 'https://www.baseball-reference.com' + str(row[1])
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    #grabbing number of MLB seasons
    for tag in soup.find_all('tfoot'):
        for tag2 in tag.find_all('tr', class_='mlb'):
            if 'seasons' in tag2.text:
                print(tag2.text.split('(')[1].split(' ')[0])

    #grabbing minors seasons
    for tag in soup.find_all('tfoot'):
        if tag.find('tr', class_='minors'):
            for tag2 in tag.find('tr', class_='minors'):
                if 'seasons' in tag2.text:
                    print(tag2.text.split('(')[1].split(' ')[0])

2
1
2
7
8
1
3
1
7
1
5
1
1
6
1
7
2
1
1
1
1
6
1
2
1
6
9
1
8
8
8
2
1
1
1
1
1
5
1
6
5
1
1
9
1
7
7
6
9
1
6
1
5
1
9
3
3
1
8
7
1
4
1
3
9
5
9
1
1
2
1
9
1
1
1
2
1
6
1
1
7
3
1
1
9
2
1
5
1
3
1
7
4
1
5
1
2
8
1
2
1
1
1
1
1
1
1
1
6
1
9
1
6
1
1
1
1
6
1
1
3
1
5
1
1
1
7
7
9
5
1
3
1
9
1


KeyboardInterrupt: 

In [852]:
url

'https://www.baseball-reference.com/register/player.fcgi?id=barnes002lar'

In [919]:
def grab_seasons(df):
    """
    Arguments: a dataframe that already has player names and the url path to their stats page on baseball-reference.com
    Returns: returns the dataframe with two more data points: # of seasons in MLB and # of seasons in Minor leagues
    """
    
    #looping through player stat pages and grabbing the HTML
    for index,row in enumerate(df.values.tolist()):
        url = 'https://www.baseball-reference.com' + str(row[1])
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")

        #grabbing number of MLB seasons
        for tag in soup.find_all('tfoot'):
            for tag2 in tag.find_all('tr', class_='mlb'):
                if 'season' in tag2.text:
                    df.at[index,'MLB_seasons'] = tag2.text.split('(')[1].split(' ')[0]

        #grabbing minors seasons
        for tag3 in soup.find_all('tfoot'):
            if tag3.find('tr', class_='minors'):
                for tag4 in tag3.find('tr', class_='minors'):
                    if 'season' in tag4.text:
                        df.at[index,'MINORS_seasons'] = tag4.text.split('(')[1].split(' ')[0]
                        
        print(str(index) + " completed!")
                        
    return df

In [920]:
#running the above function
mlb_all = grab_seasons(mlb_only)

0 completed!
1 completed!
2 completed!
3 completed!
4 completed!
5 completed!
6 completed!
7 completed!
8 completed!
9 completed!
10 completed!
11 completed!
12 completed!
13 completed!
14 completed!
15 completed!
16 completed!
17 completed!
18 completed!
19 completed!
20 completed!
21 completed!
22 completed!
23 completed!
24 completed!
25 completed!
26 completed!
27 completed!
28 completed!
29 completed!
30 completed!
31 completed!
32 completed!
33 completed!
34 completed!
35 completed!
36 completed!
37 completed!
38 completed!
39 completed!
40 completed!
41 completed!
42 completed!
43 completed!
44 completed!
45 completed!
46 completed!
47 completed!
48 completed!
49 completed!
50 completed!
51 completed!
52 completed!
53 completed!
54 completed!
55 completed!
56 completed!
57 completed!
58 completed!
59 completed!
60 completed!
61 completed!
62 completed!
63 completed!
64 completed!
65 completed!
66 completed!
67 completed!
68 completed!
69 completed!
70 completed!
71 completed!
72

554 completed!
555 completed!
556 completed!
557 completed!
558 completed!
559 completed!
560 completed!
561 completed!
562 completed!
563 completed!
564 completed!
565 completed!
566 completed!
567 completed!
568 completed!
569 completed!
570 completed!
571 completed!
572 completed!
573 completed!
574 completed!
575 completed!
576 completed!
577 completed!
578 completed!
579 completed!
580 completed!
581 completed!
582 completed!
583 completed!
584 completed!
585 completed!
586 completed!
587 completed!
588 completed!
589 completed!
590 completed!
591 completed!
592 completed!
593 completed!
594 completed!
595 completed!
596 completed!
597 completed!
598 completed!
599 completed!
600 completed!
601 completed!
602 completed!
603 completed!
604 completed!
605 completed!
606 completed!
607 completed!
608 completed!
609 completed!
610 completed!
611 completed!
612 completed!
613 completed!
614 completed!
615 completed!
616 completed!
617 completed!
618 completed!
619 completed!
620 comple

1094 completed!
1095 completed!
1096 completed!
1097 completed!
1098 completed!
1099 completed!
1100 completed!
1101 completed!
1102 completed!
1103 completed!
1104 completed!
1105 completed!
1106 completed!
1107 completed!
1108 completed!
1109 completed!
1110 completed!
1111 completed!
1112 completed!
1113 completed!
1114 completed!
1115 completed!
1116 completed!
1117 completed!
1118 completed!
1119 completed!
1120 completed!
1121 completed!
1122 completed!
1123 completed!
1124 completed!
1125 completed!
1126 completed!
1127 completed!
1128 completed!
1129 completed!
1130 completed!
1131 completed!
1132 completed!
1133 completed!
1134 completed!
1135 completed!
1136 completed!
1137 completed!
1138 completed!
1139 completed!
1140 completed!
1141 completed!
1142 completed!
1143 completed!
1144 completed!
1145 completed!
1146 completed!
1147 completed!
1148 completed!
1149 completed!
1150 completed!


In [922]:
mlb_all

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,Height,Weight,Born,Debut,Draft,School,Debut_Age,MLB_seasons,MINORS_seasons,MLB_AB_per_season
0,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,73,195,1974-07-23,2001-04-11,69,1,26.736986,2,11,39.000000
1,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,71,175,1974-08-01,1998-05-17,42,1,23.808219,2,7,109.000000
2,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,72,195,1971-09-24,2001-05-09,9,1,29.643836,8,16,48.750000
3,Edgard Clemente,/register/player.fcgi?id=clemen001edg,973,3586,501,963,125,496,285,875,...,71,188,1975-12-15,1998-09-10,10,0,22.753425,3,10,85.666667
4,Steve Decker,/register/player.fcgi?id=decker001ste,913,3228,458,975,101,575,412,470,...,75,205,1965-10-25,1990-09-18,21,1,24.915068,7,11,98.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,Geoff Blum,/register/player.fcgi?id=blum--001geo,602,2103,302,564,28,260,264,302,...,75,220,1973-04-26,1999-08-09,7,1,26.304110,14,11,283.285714
1147,Wilton Guerrero,/register/player.fcgi?id=guerre001wil,732,2816,404,877,11,249,174,375,...,71,145,1974-10-24,1996-09-03,0,0,21.876712,8,10,209.750000
1148,Mike Mordecai,/register/player.fcgi?id=mordec001mic,564,2007,299,538,33,227,212,306,...,71,175,1967-12-13,1994-05-08,33,1,26.419178,12,9,113.500000
1149,Terry Jones,/register/player.fcgi?id=jones-006ter,720,2782,449,763,4,193,225,482,...,70,165,1971-02-15,1996-09-08,40,1,25.580822,5,8,106.000000


In [923]:
#looking through the data
mlb_all['MLB_AB_per_season'] = mlb_all['MLB_AB']/mlb_all['MLB_seasons']

/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [924]:
#making sure the data looks right
mlb_all[['player_name','url_path','MLB_AB','MLB_seasons','MLB_AB_per_season']].sort_values(by='MLB_AB_per_season',ascending=False).head(50)

,player_name,url_path,MLB_AB,MLB_seasons,MLB_AB_per_season
885,Carlos Lee,/register/player.fcgi?id=lee---004car,7983,14,570.214286
785,Michael Young,/register/player.fcgi?id=young-002mic,7918,14,565.571429
723,Albert Pujols,/register/player.fcgi?id=pujols001jos,10687,19,562.473684
568,Derek Jeter,/register/player.fcgi?id=jeter-001der,11195,20,559.750000
849,Cal Ripken Jr.,/register/player.fcgi?id=ripken001cal,11551,21,550.047619
601,Jimmy Rollins,/register/player.fcgi?id=rollin001jam,9294,17,546.705882
939,Craig Biggio,/register/player.fcgi?id=biggio001cra,10876,20,543.800000
952,Johnny Damon,/register/player.fcgi?id=damon-002joh,9736,18,540.888889
280,Juan Pierre,/register/player.fcgi?id=pierre001jua,7525,14,537.500000
817,Alex Rios,/register/player.fcgi?id=rios--001ale,6419,12,534.916667


In [925]:
mlb_only

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,Height,Weight,Born,Debut,Draft,School,Debut_Age,MLB_seasons,MINORS_seasons,MLB_AB_per_season
0,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,73,195,1974-07-23,2001-04-11,69,1,26.736986,2,11,39.000000
1,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,71,175,1974-08-01,1998-05-17,42,1,23.808219,2,7,109.000000
2,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,72,195,1971-09-24,2001-05-09,9,1,29.643836,8,16,48.750000
3,Edgard Clemente,/register/player.fcgi?id=clemen001edg,973,3586,501,963,125,496,285,875,...,71,188,1975-12-15,1998-09-10,10,0,22.753425,3,10,85.666667
4,Steve Decker,/register/player.fcgi?id=decker001ste,913,3228,458,975,101,575,412,470,...,75,205,1965-10-25,1990-09-18,21,1,24.915068,7,11,98.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,Geoff Blum,/register/player.fcgi?id=blum--001geo,602,2103,302,564,28,260,264,302,...,75,220,1973-04-26,1999-08-09,7,1,26.304110,14,11,283.285714
1147,Wilton Guerrero,/register/player.fcgi?id=guerre001wil,732,2816,404,877,11,249,174,375,...,71,145,1974-10-24,1996-09-03,0,0,21.876712,8,10,209.750000
1148,Mike Mordecai,/register/player.fcgi?id=mordec001mic,564,2007,299,538,33,227,212,306,...,71,175,1967-12-13,1994-05-08,33,1,26.419178,12,9,113.500000
1149,Terry Jones,/register/player.fcgi?id=jones-006ter,720,2782,449,763,4,193,225,482,...,70,165,1971-02-15,1996-09-08,40,1,25.580822,5,8,106.000000


In [926]:
#saving this new df
with open('mlb_only_full.pkl', 'wb') as handle:
    pickle.dump(mlb_only, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [958]:
#trimming features we already know we are not going to use

#option 1 with MLB AB per season being the target variable
df2 = mlb_only[['MINORS_OPS','MLB_AB_per_season', 'MINORS_R_per_AB',
       'MINORS_SO_per_AB', 'Position', 'Bats', 'Height',
       'Weight', 'Draft', 'School', 'Debut_Age']]

#option 2 with MLB AB total being the target variable
df3 = mlb_only[['MINORS_OPS','MLB_AB', 'MINORS_R_per_AB',
       'MINORS_SO_per_AB', 'Position', 'Bats', 'Height',
       'Weight', 'Draft', 'School', 'Debut_Age']]

In [959]:
#formatting these two new dataframes
df2.index=mlb_only['player_name']
df3.index=mlb_only['player_name']

In [960]:
df2

,MINORS_OPS,MLB_AB_per_season,MINORS_R_per_AB,MINORS_SO_per_AB,Position,Bats,Height,Weight,Draft,School,Debut_Age
player_name,,,,,,,,,,,
Larry Barnes,0.833,39.000000,0.156048,0.205326,First Baseman,Left,73,195,69,1,26.736986
Justin Baughman,0.660,109.000000,0.150238,0.164149,Shortstop,Right,71,175,42,1,23.808219
Jamie Burke,0.725,48.750000,0.131893,0.131434,Catcher,Right,72,195,9,1,29.643836
Edgard Clemente,0.773,85.666667,0.139710,0.244004,Outfielder,Right,71,188,10,0,22.753425
Steve Decker,0.841,98.285714,0.141884,0.145601,Catcher,Right,75,205,21,1,24.915068
...,...,...,...,...,...,...,...,...,...,...,...
Geoff Blum,0.728,283.285714,0.143604,0.143604,Third Baseman,Both,75,220,7,1,26.304110
Wilton Guerrero,0.740,209.750000,0.143466,0.133168,Second Baseman,Both,71,145,0,0,21.876712
Mike Mordecai,0.718,113.500000,0.148979,0.152466,Shortstop,Right,71,175,33,1,26.419178


**I had to redo some of the earlier data cleaning and feature engineering I only applied to dummy_df. So that's why you see some repeated code now**

In [961]:
df2 = pd.get_dummies(df2)
df3 = pd.get_dummies(df3)

In [962]:
df2['bmi'] = (df2.Weight/2.205)/((df2.Height/39.37)**2)
df3['bmi'] = (df3.Weight/2.205)/((df3.Height/39.37)**2)

In [963]:
df2 = df2.drop(['Height', 'Weight'], axis=1)
df3 = df3.drop(['Height', 'Weight'], axis=1)


In [965]:
df2.rename(columns={"Position_Third Baseman":"Position_Third_Baseman", "Position_First Baseman": "Position_First_Baseman", "Position_Second Baseman": "Position_Second_Baseman"}, inplace=True)
df3.rename(columns={"Position_Third Baseman":"Position_Third_Baseman", "Position_First Baseman": "Position_First_Baseman", "Position_Second Baseman": "Position_Second_Baseman"}, inplace=True)

In [985]:
df2.loc['Alfonso Soriano'] = df2.loc['Alfonso Soriano'].fillna(0.791)
df3.loc['Alfonso Soriano'] = df3.loc['Alfonso Soriano'].fillna(0.791)

**Ok done! Now time to save the data and proceed to EDA, Regularization and Validation**

In [987]:
with open('per_season_mlb_eda_ready.pkl', 'wb') as handle:
    pickle.dump(df2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [988]:
with open('per_ab_mlb_eda_ready.pkl', 'wb') as handle:
    pickle.dump(df3, handle, protocol=pickle.HIGHEST_PROTOCOL)